In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

# 1. Basic about the graph and Linear Regression:
- Note: All variable are dropped after each graph run

In [9]:
# gen data
X = np.random.randn(20,2)
y = 2*X[:,0] - 5*X[:,1] + 3 + np.random.randn(20,)
y = y.reshape(20,1)

In [10]:
X_tensor = tf.constant(X, name='X')
y_tensor = tf.constant(y, name='y')

In [11]:
y_tensor.dtype

tf.float64

In [143]:
W = tf.Variable(tf.random_uniform([2,1], -1,1, dtype=tf.float64), name='theta')
b = tf.Variable(tf.random_uniform([1], -1, 1, dtype=tf.float64), name='bias')

In [146]:
lr = 0.001
n_epochs = 10000

y_pred = tf.add(tf.matmul(X_tensor, W), b, name = 'prediction')
error = y_pred - y_tensor
loss = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/20 * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(W, W-lr*gradients)

In [147]:
# perform linear regression
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(n_epochs):
        if(i%1000==0):
            print('loss at iter %d: %f'%(i, loss.eval()))
        sess.run(training_op)

loss at iter 0: 48.613830
loss at iter 1000: 7.238812
loss at iter 2000: 6.779519
loss at iter 3000: 6.774283
loss at iter 4000: 6.774221
loss at iter 5000: 6.774221
loss at iter 6000: 6.774221
loss at iter 7000: 6.774221
loss at iter 8000: 6.774221
loss at iter 9000: 6.774221


In [119]:
# using autodiff
gradients_auto = tf.gradients(loss, W)[0]
training_op_auto = tf.assign(W, W-lr*gradients_auto)

# perform linear regression
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(n_epochs):
        if(i%1000==0):
            print('loss at iter %d: %f'%(i, loss.eval()))
        sess.run(training_op_auto)

loss at iter 0: 46.654586
loss at iter 1000: 8.799842
loss at iter 2000: 8.373501
loss at iter 3000: 8.368555
loss at iter 4000: 8.368496
loss at iter 5000: 8.368495
loss at iter 6000: 8.368495
loss at iter 7000: 8.368495
loss at iter 8000: 8.368495
loss at iter 9000: 8.368495


# 2. Feed dict:
- A placeholder to put value into in run time

In [153]:
lr = 0.01
n_epochs = 1000

In [154]:
X_feed = tf.placeholder(shape=(20,2), dtype=tf.float64, name='X_feed')
y_feed = tf.placeholder(shape=(20,1), dtype=tf.float64, name='y_feed')

In [155]:
y.shape

(20, 1)

In [156]:
error = tf.add(tf.matmul(X_feed, W),b) - y_feed 
loss = tf.reduce_mean(tf.square(error), name='mse')
gradients = tf.gradients(loss, [W])[0]
training_op = tf.assign(W, W-lr*gradients)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(n_epochs):
        if(i%100==0):
            print('loss at iter %d: %f'%(i, loss.eval(feed_dict={X_feed: X, y_feed:y})))
        sess.run(training_op, feed_dict={X_feed: X, y_feed:y})

loss at iter 0: 49.712270
loss at iter 100: 8.952125
loss at iter 200: 8.474871
loss at iter 300: 8.469060
loss at iter 400: 8.468986
loss at iter 500: 8.468985
loss at iter 600: 8.468985
loss at iter 700: 8.468985
loss at iter 800: 8.468985
loss at iter 900: 8.468985


# 3. Using an optimizer:
- Instead of using your own training op and assign value to W after each epoch manually, we use Opitmizers

In [158]:
lr = 0.01
n_epochs = 1000

In [7]:
X_feed = tf.placeholder(shape=(20,2), dtype=tf.float64, name='X_feed')
y_feed = tf.placeholder(shape=(20,1), dtype=tf.float64, name='y_feed')

W = tf.Variable(tf.random_uniform([2,1], -1,1, dtype=tf.float64), name='theta')
b = tf.Variable(tf.random_uniform([1], -1, 1, dtype=tf.float64), name='bias')

In [12]:
error = tf.add(tf.matmul(X_feed, W),b) - y_feed 
loss = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate = lr)
training_op = optimizer.minimize(loss)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(n_epochs):
        if(i%100==0):
            print('loss at iter %d: %f'%(i, loss.eval(feed_dict={X_feed: X, y_feed:y})))
        sess.run(training_op, feed_dict={X_feed: X, y_feed:y})

loss at iter 0: 34.167995
loss at iter 100: 1.221681
loss at iter 200: 0.618817
loss at iter 300: 0.581512
loss at iter 400: 0.577819
loss at iter 500: 0.577425
loss at iter 600: 0.577383
loss at iter 700: 0.577378
loss at iter 800: 0.577378
loss at iter 900: 0.577378


# 4. Visulize training loss and graph:
- Log loss every epoch using tf.summary
- write tf.get_default_graph() to file
- visualize them on tensorboard

In [27]:
lr = 0.01
n_epochs = 1000

tf.reset_default_graph()

X_feed = tf.placeholder(shape=(20,2), dtype=tf.float64, name='X_feed')
y_feed = tf.placeholder(shape=(20,1), dtype=tf.float64, name='y_feed')

W = tf.Variable(tf.random_uniform([2,1], -1,1, dtype=tf.float64), name='theta')
b = tf.Variable(tf.random_uniform([1], -1, 1, dtype=tf.float64), name='bias')

error = tf.add(tf.matmul(X_feed, W),b) - y_feed 
loss = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate = lr)
training_op = optimizer.minimize(loss)

In [28]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_dir = "tf_logs"
log_dir = "{}/run-{}/".format(root_dir, now)

# declare loss logger
loss_summary = tf.summary.scalar('MSE', loss)
# declare file writer and save current default graph
file_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

In [29]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(n_epochs):
        if(i%100==0):
            print('loss at iter %d: %f'%(i, loss.eval(feed_dict={X_feed: X, y_feed:y})))
            # get summary string
            summary_string = loss_summary.eval(feed_dict={X_feed: X, y_feed:y})
            # write current loss to file
            file_writer.add_summary(summary_string, i)
        sess.run(training_op, feed_dict={X_feed: X, y_feed:y})
        
file_writer.close()

loss at iter 0: 54.967268
loss at iter 100: 1.635727
loss at iter 200: 0.639550
loss at iter 300: 0.583268
loss at iter 400: 0.577996
loss at iter 500: 0.577444
loss at iter 600: 0.577385
loss at iter 700: 0.577378
loss at iter 800: 0.577378
loss at iter 900: 0.577378


In [ ]:
# 4. Name scope and shared variable
- Name scope is used to group variables which are belongs to the same computational task
- You can create tensor inside a namescope and its name will have prefix which is that namescope
- variable_scope acts like namescope but allow you to use shared variable
- Shared variables can be create or reuse by calling tf.get_variable()
    - Only variables created by tf.get_variable() can be reused
    - If want to reuse, define reuse = True at their variable scope

In [61]:
tf.reset_default_graph()

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
log_dir = "log_dirs/run-{}".format(now)



with tf.name_scope('my_var1'):
    x = tf.Variable(4, name='x')
    y = tf.Variable(5, name='y')
    z = tf.add(x,y,name='z')
    
with tf.name_scope('my_var2'):
    t = tf.add(x,y, name='t')
    
with tf.variable_scope('my_shared'):
    shared_m = tf.get_variable(initializer=tf.constant(np.random.randn(3,2)), name='shared_m' )
    
with tf.variable_scope('my_shared', reuse=True):
    shared_t = tf.get_variable(name='shared_m', dtype=tf.float64)
    
with tf.name_scope('my_var2'):
    k = tf.reduce_mean(tf.square(shared_t), name='k')
    
file_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph()) 
    
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(z.eval())
    print(t.eval())
    
file_writer.close()

9
9
